In [1]:
#grep -e "PATMAT" -e "#" 10x22.vcf | grep -v "0/1" > 10x22_ptmt.vcf
import numpy as np
chrom_output=10
vcf_file_address = 'data/'+str(chrom_output)+'/out.vcf' #tst.vcf' #
vcf_file = open(vcf_file_address,'r')

hap_blocks={} # key: id of phase block (i.e. phase st), value=allele_1 
idc_blocks={} # key: id of phase block (i.e. phase st), value=var_idx   

phased_vcf_dic={}

var_pos_list=[] # all blocks consequently 
header_lines_list=[] 

var_idx=0 # for the first variant, it's 1. 1-based indexing 
for line in vcf_file:
    line_strip=line.strip() 
    if line_strip.startswith('#'):
        header_lines_list.append(line_strip)
        sample_names=line_strip.split('\t')[9:11]# last line of header contains sample name
    else:
        var_idx+=1
        line_parts=line_strip.split('\t') 
        
        gt_flags, sample1=line_parts[8:10] # sample of interest that is merged        
        chrom=line_parts[0]
        
        if str(chrom_output)!=chrom:
            print(chrom_output,chrom)
        
        var_pos=int(line_parts[1])
        var_pos_list.append(var_pos)
        #if var_pos < 7398830: #60780
            
        phased_vcf_dic[var_idx]=line_parts


        sample1_split=sample1.split(":")
        #gt_flags_split=gt_flags.split(":")
        gt_id_flags=gt_flags.split(":").index("GT")
        ps_id_flags=gt_flags.split(":").index("PS")
        allele_sample1=sample1_split[gt_id_flags]        
        if '.' not in allele_sample1:
            block_id1=int(sample1_split[ps_id_flags])
            if block_id1 in hap_blocks:
                hap_blocks[block_id1].append(allele_sample1) # append new variants to the existing phase block
                idc_blocks[block_id1].append(var_idx)
            else:
                hap_blocks[block_id1]=[allele_sample1] # creat new phase block
                idc_blocks[block_id1]=[var_idx]

        else:
            print('There is a vriant that ONT is not phased.. removed it first')


print('number of blocks in ont',len(hap_blocks))

# print('ont')
# for block_id, pos_block in pos_blocks.items():
#     print(block_id,len(pos_block))
# print('ont')
for block_id1, hap_block  in hap_blocks.items():
    print(block_id1,len(hap_block))
hap_block[:2]


number of blocks in ont 22
63828 22109
18098887 11005
27607877 11132
38869204 3743
42354942 1293
42596699 5413
46486766 106
46666247 181
46787270 1854
47532101 413
47894257 270
48108270 1137
49088577 8
49196518 1785
51215194 172
51449190 389
51722995 68171
125922555 3004
128766436 6149
133431868 342
133730171 507
133987652 2598


['1|0', '1|0']

In [2]:
#### Parsing pusod read
# handling pop information reporting only once

# reading short haplotype block file
pseudo='data/'+str(chrom_output)+'/pairs.txt'
file_hap = open(pseudo,'r'); 
pop_inf_dic={} # key variant index, value two lists 
for line in file_hap:
    line_parts=line.strip().split('\t')     # ['42081', '0', '42096', '0']
    snp1_idx=int(line_parts[0])
    allele1_snp1=int(line_parts[1])
    snp2_idx=int(line_parts[2])
    allele1_snp2=int(line_parts[3])
    #pairs_list.append([snp1_idx,allele1_snp1,snp2_idx,allele1_snp2])
    
    snps_idx=[snp1_idx,snp2_idx]
    snps_allele=[allele1_snp1,allele1_snp2]
    host_idx=max(snps_idx)
    guest_idx=min(snps_idx)
    if host_idx not in pop_inf_dic.keys():
        pop_inf_dic[host_idx]={'sim':list(),'dis':list()}
    if allele1_snp1==allele1_snp2:
        pop_inf_dic[host_idx]['sim'].append(guest_idx)  # sim shows the relation between two elements of a pop pair
    else:
        pop_inf_dic[host_idx]['dis'].append(guest_idx) 

# line_parts
# print(line_parts,'\n', pairs_list[0])
# print(len(pairs_list))
print(pop_inf_dic[host_idx])
print(len(pop_inf_dic))



{'sim': [141700, 141716], 'dis': []}
78486


In [3]:
# comparing one ONT block with all pop information

# block_id1=8889839 # test for chr 19
# hap_block=hap_blocks[block_id1]
# idc_block=idc_blocks[block_id1]

# if 1:
def compare_ont_pop(hap_block,idc_block,pop_inf_dic,interval):
    
    # input: interval is based on var_i 
    # output: is ont_pop_block[var_idx]=ont_pop 
    
    #var_i is the index of variant within block
    #var_idx is the index of vriant globally in the VCF file     
    
    ont_pop_block={}
    min_interval=min(list(interval))
    max_interval=max(list(interval))
    interval_idx=set(idc_block[min_interval:max_interval]) # set is faster than list for check if element is in it
    for var_i in interval: # the ONT hap block
        alleles=hap_block[var_i]
        var_idx=idc_block[var_i]
        
        #var_i is the index of variant within block
        #var_idx is the index of vriant globally in the VCF file 
        
        
        if var_idx in pop_inf_dic.keys():
            list_pairs=pop_inf_dic[var_idx]
            match_sim_list=[]
            match_dis_list=[]
            mismatch_sim_list=[]
            mismatch_dis_list=[]
            for sim_idx in pop_inf_dic[var_idx]['sim']: # sim shows the relation between two elements of a pop pair
                
                #if sim_idx in idc_block: # if the SNP  of pairs is in this ONT block
                try:
                    alleles_pop=hap_block[idc_block.index(sim_idx)] # extract the allele of the snp 'sim' which is the other snp in pair
                    if sim_idx in interval_idx:
                        if alleles_pop==alleles: 
                            match_sim_list.append(str(sim_idx))  
                        else:
                            mismatch_sim_list.append(str(sim_idx))     
                except:
                    pass
                        
            for sim_idx in pop_inf_dic[var_idx]['dis']: # sim shows the relation between two elements of a pop pair
                #if sim_idx in idc_block: # if the SNP  of pairs is in this ONT block
                try:
                    alleles_pop=hap_block[idc_block.index(sim_idx)] # extract the allele of the snp 'sim' which is the other snp in pair
                    if sim_idx in interval_idx:
                        if alleles_pop==str(1-int(alleles[0]))+'|'+str(1-int(alleles[2])):  
                            match_sim_list.append(str(sim_idx))  
                        else:
                            mismatch_sim_list.append(str(sim_idx))            
                except:
                    pass
        
            match_list=match_sim_list+match_dis_list
            mismatch_list=mismatch_sim_list+mismatch_dis_list
            ont_pop=[match_list,mismatch_list]
        else:
            ont_pop=[[],[]]

        ont_pop_block[var_idx]=ont_pop
    #print('comparison is done')
    return  ont_pop_block
    

In [4]:

# comparing ONT (hap_blocks_sample1_dic)with pop (short_blocks_dic_pos)

ont_pop_blocks={}
for block_id1, hap_block in hap_blocks.items(): # all ONT hap blocks  
    idc_block=idc_blocks[block_id1]
     
    #if block_id1==27731803: #27731803: 60780 20588536 8889839
    ont_pop_block=compare_ont_pop(hap_block,idc_block,pop_inf_dic,range(len(hap_block)))
    ont_pop_blocks[block_id1]=ont_pop_block

#hap_blocks.keys()
#var_pos_list[31566-1] #grep -v "#" out.vcf | sed -n 31566,31566p
#ont_pop_blocks[60780][31576]
# print(ont_pop_blocks[60780][292])
# print('for variant',var_pos_list[292-1],'we have',var_pos_list[268-1],var_pos_list[269-1],var_pos_list[261-1])


In [ ]:
#aa=0


#block_id1=27731803 # 60780  # 20588536 27731803 8889839
#hap_block=hap_blocks[block_id1]

#if 1:
flip_list=[]
cut_dic={}



for block_id1, hap_block in hap_blocks.items():
    cut_dic[block_id1]=[]

    idc_block=idc_blocks[block_id1]
    ont_pop_block=ont_pop_blocks[block_id1]

    for var_i, var_idx in enumerate(idc_block): 
        #var_i is the index of variant within block
        #var_idx is the index of vriant globally in the VCF file 

        ont_pop = ont_pop_block[var_idx]  # after a cut, it may use a newr version of this vsrisble
        if ont_pop==[[],[]]:
            out_pairs='.'
        else:
            [match_list,mismatch_list]=ont_pop    
            len_mismatch_list=len(mismatch_list)
            len_match_list=len(match_list)
            condition=0
            condition_flip=0
            condition_cut=0
            if len_mismatch_list>=2:
                if len_match_list==0:
                    condition=1
                elif len_mismatch_list/len_match_list >= 0.75 :
                    condition=1
            if condition:
                #print(len_mismatch_list,var_idx,var_pos_list[var_idx-1])
                ont_pop_prev = ont_pop_block[var_idx-1] 
                ont_pop_nex = ont_pop_block[var_idx+1] 

                len_match_list_nex=len(ont_pop_nex[0])
                len_mismatch_list_nex=len(ont_pop_nex[1])
                len_match_list_prev=len(ont_pop_prev[0])
                len_mismatch_list_prev=len(ont_pop_prev[1])

                if len_match_list_nex>1.3*len_mismatch_list_nex and len_match_list_prev>1.3*len_mismatch_list_prev: #  ?? if no pop information in last var
                    condition_flip=1
                if len_match_list_prev==0:
                    if  len_mismatch_list_nex>=2:
                        condition_cut=1
                elif len_mismatch_list_nex/len_match_list_prev >= 0.75:
                    condition_cut=1

            if condition_flip:
                print('flip candidate', var_idx, var_pos_list[var_idx-1])
                flip_list.append(var_idx)
            if condition_cut and not condition_flip:
                cut_pos=var_pos_list[var_idx-1] # cut_pos the starting position of new block
                print('cut candidate', cut_pos,var_i,var_idx)   
                cut_dic[block_id1].append(var_idx)
                interval=range(var_i,len(hap_block))
                

                
                
                ont_pop_block=compare_ont_pop(hap_block,idc_block,pop_inf_dic,interval)
#                 if cut_pos==43529026:
#                     print('after')
#                     print(ont_pop_block[49889])
#                     print(ont_pop_block[49900])
#                     break
                    
                    
                    

            

cut candidate 4597794 5686 5687
cut candidate 4601068 5690 5691
flip candidate 9579 7346152
cut candidate 12893528 16417 16418
cut candidate 13263027 16944 16945
flip candidate 17377 13562076
flip candidate 17836 14169317
flip candidate 20744 16734165
flip candidate 23432 19293071
cut candidate 19874459 2234 24344
flip candidate 24379 20058500
cut candidate 24604635 6286 28396
flip candidate 28417 24624556
cut candidate 30678263 3622 36737
cut candidate 33843524 6463 39578
cut candidate 48888592 972 58492
cut candidate 53176928 1882 62893
cut candidate 65836969 16461 77472
cut candidate 67202386 17809 78820
cut candidate 67225139 17817 78828
flip candidate 81115 68697656
cut candidate 72574945 23810 84821
cut candidate 73731442 25059 86070


In [6]:
# improving flips
# for block_id1, hap_block in hap_blocks.items():
#     idc_block=idc_blocks[block_id1]
#     ont_pop_block=ont_pop_blocks[block_id1]

phased_vcf_dic_improved=phased_vcf_dic
for var_idx in flip_list:
    line_parts=phased_vcf_dic[var_idx]
    gt_flags, sample1=line_parts[8:10]
    sample1_split=sample1.split(":")
    gt_id_flags=gt_flags.split(":").index("GT")        
    allele_sample1=sample1_split[gt_id_flags]   
    
    allele_sample1_flp=str(1-int(allele_sample1[0]))+'|'+str(1-int(allele_sample1[2]))
    sample1_split_flp=sample1_split
    sample1_split_flp[gt_id_flags]=allele_sample1_flp
    line_parts_flp=line_parts
    line_parts_flp[9]=':'.join(sample1_split_flp)
    phased_vcf_dic_improved[var_idx]=line_parts_flp
    #block_id1=int(sample1_split[ps_id_flags])
    #ps_id_flags=gt_flags.split(":").index("PS")

#changing block is done in reporting 
    

In [7]:
# for block_id1, hap_block in hap_blocks.items():

#     idc_block=idc_blocks[block_id1]
#     ont_pop_block=ont_pop_blocks[block_id1]

#     for var_i, var_idx in enumerate(idc_block): 

#             cut_block=cut_dic[block_id1]


#                 new_blockid=cut_pos
#                 dic1{}

In [8]:
# block_id1=60780
# new_blockid_dic={}
# cut_block=cut_dic[block_id1]
# cut_block_complete=[idc_block[0]]+cut_block+[idc_block[-1]] # including start and end var_idx of block
# cut_block_complete
# cut_dic

In [17]:

# creating new block id dic

new_blockid_dic={}

for block_id1 in hap_blocks.keys():

    cut_block=cut_dic[block_id1]
    idc_block=idc_blocks[block_id1]

    cut_block_complete=[idc_block[0]]+cut_block+[idc_block[-1]] # including start and end var_idx of block

    for i in range(1,len(cut_block_complete)):
        pre_cut_idx=cut_block_complete[i-1]
        next_cut_idx=cut_block_complete[i]

        blockid=var_pos_list[pre_cut_idx-1]
        for k in range(pre_cut_idx,next_cut_idx+1):
            new_blockid_dic[k]=blockid



print(len(new_blockid_dic))



71453


In [19]:
a=sorted(new_blockid_dic.keys())
print(a[0],a[-1])


for block_id1 in hap_blocks.keys():
    print(block_id1)

1 71453
60780
7398833
8737935
8889839
20588536
27731803


In [20]:
#dic1={1:1}


phased_vcf_dic_improved2=phased_vcf_dic_improved

sorted_var_indx=sorted(phased_vcf_dic_improved.keys())

cut_dic_remain=cut_dic

for var_indx in sorted_var_indx: # imprvd_phased_vcf_dic
    
    #if var_indx in dic1: # those variant that we need to change their block id 
    line_parts=phased_vcf_dic_improved[var_indx]
    gt_flags, sample1=line_parts[8:10]
    sample1_split=sample1.split(":")
    ps_id_flags=gt_flags.split(":").index("PS")
    block_id1=int(sample1_split[ps_id_flags])

    new_blockid=str(new_blockid_dic[var_indx])


    #cut_dic[block_id1]
    sample1_split_changed=sample1.split(":")
    sample1_split_changed[ps_id_flags]=new_blockid
    line_parts_changed=line_parts
    line_parts_changed[9]=':'.join(sample1_split_changed)
    phased_vcf_dic_improved2[var_indx]=line_parts_changed



In [ ]:
# max(phased_vcf_dic_improved.keys())

In [ ]:
# line_parts=phased_vcf_dic[10]
# gt_flags, sample1=line_parts[8:10]
# sample1_split=sample1.split(":")
# ps_id_flags=gt_flags.split(":").index("PS")
# block_id1=int(sample1_split[ps_id_flags])
# block_id1


In [ ]:
# phased_vcf_dic_improved=phased_vcf_dic
# for var_idx in cut_set:
#     line_parts=phased_vcf_dic[var_idx]
#     gt_flags, sample1=line_parts[8:10]
#     sample1_split=sample1.split(":")        
#     gt_id_flags=gt_flags.split(":").index("GT")        
#     allele_sample1=sample1_split[gt_id_flags]   
#     block_id1=int(sample1_split[ps_id_flags])
#     ps_id_flags=gt_flags.split(":").index("PS")
    
    
    
#     allele_sample1_flp=str(1-int(allele_sample1[0]))+'|'+str(1-int(allele_sample1[2]))
#     sample1_split_flp=sample1_split
#     sample1_split_flp[gt_id_flags]=allele_sample1_flp
#     line_parts_flp=line_parts
#     line_parts_flp[9]=':'.join(sample1_split_flp)
#     phased_vcf_dic_improved[var_idx]=line_parts_flp


In [21]:



vcf_file_improved_address = 'data/'+str(chrom_output)+'/out_improved.vcf' #tst.vcf' #

# reading phased VCF file,  use grep before that, only 0|1 or 1|0. bi -allelic
vcf_file_improved = open(vcf_file_improved_address,'w');  # phased_vcf_dic

sorted_var_indx=sorted(phased_vcf_dic_improved2.keys())

for header_line in header_lines_list:
    vcf_file_improved.write(header_line+'\n')
for var_indx in sorted_var_indx: # imprvd_phased_vcf_dic
    line_parts=phased_vcf_dic_improved2[var_indx]
    vcf_file_improved.write('\t'.join(line_parts)+'\n')
vcf_file_improved.close()

In [ ]:
# # summary
# import numpy as np

# len_list=[]
# lengths=[]
# for block_id1, idc_block  in idc_blocks.items():
#     indc_srtd=sorted(idc_block)
#     diff=var_pos_list[indc_srtd[-1]-1]-var_pos_list[indc_srtd[0]-1] # all of the variant in a block are consequtive 
#     lengths.append(diff)

#     len_list.append(len(hap_block))

# all_len=sorted(lengths, reverse=True)
# csum=np.cumsum(all_len)
# n2=int(sum(lengths)/2)
# csumn2=min(csum[csum >= n2])
# ind=np.where(csum == csumn2)

# n50 = all_len[int(ind[0])]

# print('** Chr 19 ** ')
# print('Number phased heterzygous SNVs:')
# print('Number non-phased heterzygous SNVs, 0/0:','1/1:')
# print('Number homozygous SNVs:')

# print('N50 of phase blocks: ',n50)
# #print('Mean of  genomic length of haplotype 'np.mean(lengths))
# print('Mean number of SNV in phase block ',np.mean(len_list))
# print('Number conflict SNV with population information: ','%')


s